In [1]:
import pandas as pd

data = pd.read_excel(r"C:\Users\17293\Desktop\大三上\社会网络分析\数据处理_new\train_new.xlsx" )
data

,account_age,id,name_length,name,screen_name_length,screen_name,location,description,url,protected,...,profile_text_color,profile_use_background_image,has_extended_profile,default_profile,default_profile_image,following,follow_request_sent,notifications,translator_type,label
0,1761,1318284638,17,Syed Haider Naqvi,6,Hnakvi,Islamabad,NaN,NaN,False,...,darkslategray,True,False,True,False,False,False,False,none,bot
1,1759,17442457,19,Cleveland Clinic MD,11,CleClinicMD,"Cleveland, Ohio, United States",News and insights for physicians and clinical ...,https://t.co/IxMra2OEey,False,...,darkslategray,True,False,False,False,False,False,False,none,bot
2,2592,102069605,7,2NE1 TV,6,2NE1tv,Seoul,We tweet about 2NE1's video and photo. It’s 2N...,NaN,False,...,darkolivegreen,True,False,False,False,False,False,False,none,bot
3,1759,1367759732,19,Alastair de kretser,15,Alastairkretser,England,"Travel, tech and stuff 😀",NaN,False,...,darkslategray,True,True,False,False,False,False,False,none,bot
4,1829,2927879268,14,Gran Horóscopo,13,GranHoroscopo,México - Argentina,Tu mejor prediccion diaria ORIGINAL para todos...,https://t.co/rDVUTyCn9E,False,...,Darkslategray,True,False,True,False,False,False,False,regular,bot
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,2314,464353156,6,Biα ♐️,13,Biia_Amaraal1,Artur Nogueira,👻 bia.amarall/ ♐️,NaN,False,...,darkslategray,True,True,False,False,False,False,False,none,human
1982,1759,214891135,12,Brooke Marie,14,BrookeMarieCam,🤱🏼,Welcome! I specialize in Adult (18+) Custom Vi...,https://t.co/uAZ5LhUnNT,False,...,darkslategray,False,True,False,False,False,False,False,none,human
1983,1760,247977754,9,Bianca 🌴,11,biancabizb_,"Arauco, Chile",18 • 🐾💘,https://t.co/Qfnt0mOUG3,False,...,darkslategray,True,True,False,False,False,False,False,regular,human
1984,1760,274251944,6,Cameen,8,Cameen23,"New England, Brooklyn-NY",New England Music Awards Top five Artist Of Th...,https://t.co/vhcGaX7t9m,False,...,darkslategray,True,False,False,False,False,False,False,none,human


In [2]:
data.columns

Index(['account_age', 'id', 'name_length', 'name', 'screen_name_length',
       'screen_name', 'location', 'description', 'url', 'protected',
       'followers_count', 'friends_count', 'listed_count', 'favourites_count',
       'geo_enabled', 'verified', 'statuses_count', 'lang',
       'contributors_enabled', 'is_translator', 'is_translation_enabled',
       'profile_background_color', 'profile_background_image_url',
       'profile_background_tile', 'profile_image_url', 'profile_banner_url',
       'profile_link_color', 'profile_sidebar_border_color',
       'profile_sidebar_fill_color', 'profile_text_color',
       'profile_use_background_image', 'has_extended_profile',
       'default_profile', 'default_profile_image', 'following',
       'follow_request_sent', 'notifications', 'translator_type', 'label'],
      dtype='object')

In [3]:
import pandas as pd
import numpy as np
import requests
from PIL import Image
from io import BytesIO
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

df = pd.DataFrame(data)

# 加载MobileNetV2模型，包括权重
mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def extract_features_from_url(url):
    try:
        # 从URL下载图像
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))

        # 调整图像大小为MobileNetV2的输入尺寸
        img = img.resize((224, 224))

        # 转换图像为RGB格式
        img = img.convert("RGB")

        # 图像预处理
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # 使用MobileNetV2模型提取特征
        features = mobilenet_model.predict(x)

        return features.flatten()  # 将特征展平为一维数组
    except Exception as e:
        print(f"Error processing image from {url}: {e}")
        return None


处理数据中的URL

In [4]:
dev = pd.read_excel(r"C:\Users\17293\Desktop\大三上\社会网络分析\数据处理_new\dev_new.xlsx" )
test = pd.read_excel(r"C:\Users\17293\Desktop\大三上\社会网络分析\数据处理_new\test_new.xlsx")

In [5]:
df['profile_background_image_features'] = df['profile_background_image_url'].apply(extract_features_from_url)
df['profile_image_features'] = df['profile_image_url'].apply(extract_features_from_url)
df['profile_banner_features'] = df['profile_banner_url'].apply(extract_features_from_url)

df_expanded_pbif = df['profile_background_image_features'].apply(pd.Series) if 'profile_background_image_features' in df.columns else None
df_expanded_pif = df['profile_image_features'].apply(pd.Series) if 'profile_image_features' in df.columns else None
df_expanded_pbf = df['profile_banner_features'].apply(pd.Series) if 'profile_banner_features' in df.columns else None

# 合并原始数据和展开后的数据
result_df = pd.concat([df, df_expanded_pbif, df_expanded_pbf, df_expanded_pif], axis=1)
result_df = result_df.drop(columns=['profile_background_image_features', 'profile_image_features', 'profile_banner_features'])
result_df

1/1 [==============================] - 0s 63ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 63ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 56ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 60ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 60ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 56ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 50ms/step
Error processing i

,account_age,id,name_length,name,screen_name_length,screen_name,location,description,url,protected,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
0,1761,1318284638,17,Syed Haider Naqvi,6,Hnakvi,Islamabad,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1759,17442457,19,Cleveland Clinic MD,11,CleClinicMD,"Cleveland, Ohio, United States",News and insights for physicians and clinical ...,https://t.co/IxMra2OEey,False,...,0.0,0.0,0.0,0.110568,0.305912,0.000000,0.000000,0.144667,0.071325,0.000000
2,2592,102069605,7,2NE1 TV,6,2NE1tv,Seoul,We tweet about 2NE1's video and photo. It’s 2N...,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1759,1367759732,19,Alastair de kretser,15,Alastairkretser,England,"Travel, tech and stuff 😀",NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1829,2927879268,14,Gran Horóscopo,13,GranHoroscopo,México - Argentina,Tu mejor prediccion diaria ORIGINAL para todos...,https://t.co/rDVUTyCn9E,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,2314,464353156,6,Biα ♐️,13,Biia_Amaraal1,Artur Nogueira,👻 bia.amarall/ ♐️,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1982,1759,214891135,12,Brooke Marie,14,BrookeMarieCam,🤱🏼,Welcome! I specialize in Adult (18+) Custom Vi...,https://t.co/uAZ5LhUnNT,False,...,0.0,0.0,0.0,0.033084,1.803024,0.388431,0.184121,0.987681,0.023849,1.576365
1983,1760,247977754,9,Bianca 🌴,11,biancabizb_,"Arauco, Chile",18 • 🐾💘,https://t.co/Qfnt0mOUG3,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1984,1760,274251944,6,Cameen,8,Cameen23,"New England, Brooklyn-NY",New England Music Awards Top five Artist Of Th...,https://t.co/vhcGaX7t9m,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dev['profile_background_image_features'] = dev['profile_background_image_url'].apply(extract_features_from_url)
dev['profile_image_features'] = dev['profile_image_url'].apply(extract_features_from_url)
dev['profile_banner_features'] = dev['profile_banner_url'].apply(extract_features_from_url)
dev_expanded_pbif = dev['profile_background_image_features'].apply(pd.Series) if 'profile_background_image_features' in dev.columns else None
dev_expanded_pif = dev['profile_image_features'].apply(pd.Series) if 'profile_image_features' in dev.columns else None
dev_expanded_pbf = dev['profile_banner_features'].apply(pd.Series) if 'profile_banner_features' in dev.columns else None
result_dev = pd.concat([dev, dev_expanded_pbif, dev_expanded_pbf, dev_expanded_pif], axis=1)
result_dev = result_dev.drop(columns=['profile_background_image_features', 'profile_image_features', 'profile_banner_features'])


test['profile_background_image_features'] = test['profile_background_image_url'].apply(extract_features_from_url)
test['profile_image_features'] = test['profile_image_url'].apply(extract_features_from_url)
test['profile_banner_features'] = test['profile_banner_url'].apply(extract_features_from_url)
test_expanded_pbif = test['profile_background_image_features'].apply(pd.Series) if 'profile_background_image_features' in test.columns else None
test_expanded_pif = test['profile_image_features'].apply(pd.Series) if 'profile_image_features' in test.columns else None
test_expanded_pbf = dev['profile_banner_features'].apply(pd.Series) if 'profile_banner_features' in test.columns else None
result_test = pd.concat([test, test_expanded_pbif, test_expanded_pbf, test_expanded_pif], axis=1)
result_test = result_test.drop(columns=['profile_background_image_features', 'profile_image_features', 'profile_banner_features'])

1/1 [==============================] - 0s 109ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 96ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 91ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 94ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 94ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 94ms/step
Error processing image from nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
1/1 [==============================] - 0s 93ms/step
Error processing 

In [6]:
result_df.to_excel(r"C:\Users\17293\Desktop\大三上\社会网络分析\数据处理_new\train_withurl.xlsx", index=False)
result_dev.to_excel(r"C:\Users\17293\Desktop\大三上\社会网络分析\数据处理_new\dev_withurl.xlsx", index=False)
result_test.to_excel(r"C:\Users\17293\Desktop\大三上\社会网络分析\数据处理_new\test_withurl.xlsx", index=False)

In [2]:
import pandas as pd
result_df = pd.read_excel(r"E:\大三上\社会网络分析\数据处理_new\train_withurl.xlsx")
result_dev = pd.read_excel(r"E:\大三上\社会网络分析\数据处理_new\dev_withurl.xlsx")
result_test = pd.read_excel(r"E:\大三上\社会网络分析\数据处理_new\test_withurl.xlsx")

In [3]:
# 将train和dev合并，随后随机28开划分
merged_data = pd.concat([result_df, result_dev], axis=0, ignore_index=True)
merged_data = merged_data.drop(columns = ['name', 'screen_name', 'location', 'description', 'url', 'profile_background_image_url', 'profile_banner_url', 'profile_image_url'])
result_test = result_test.drop(columns = ['name', 'screen_name', 'location', 'description', 'url', 'profile_background_image_url', 'profile_banner_url', 'profile_image_url'])

In [4]:
merged_data.columns

Index([       'account_age',                 'id',        'name_length',
       'screen_name_length',          'protected',    'followers_count',
            'friends_count',       'listed_count',   'favourites_count',
              'geo_enabled',
       ...
                   '1270.2',             '1271.2',             '1272.2',
                   '1273.2',             '1274.2',             '1275.2',
                   '1276.2',             '1277.2',             '1278.2',
                   '1279.2'],
      dtype='object', length=3871)

In [5]:
# 数据处理：分类变量映射为数字特征。分离标签和特征。
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.preprocessing import StandardScaler

selected_columns = ['protected', 'geo_enabled', 'verified', 'lang', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_tile', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'translator_type', 'label']

# 初始化LabelEncoder
label_encoder = LabelEncoder()

# 对选定的列进行分类变量到数字编码的转换
merged_data[selected_columns] = merged_data[selected_columns].apply(lambda col: label_encoder.fit_transform(col))
result_test[selected_columns] = result_test[selected_columns].apply(lambda col: label_encoder.fit_transform(col))

# 数据归一化 (使用Z-score标准化)
#merged_data.columns = merged_data.columns.astype(str)
#result_test.columns = result_test.columns.astype(str)
#scaler = StandardScaler()
#merged_data = scaler.fit_transform(merged_data)
#result_test = scaler.fit_transform(result_test)
# 打印转换后的数据框
merged_data

      account_age          id  name_length  screen_name_length  protected  \
0            1761  1318284638           17                   6          0   
1            1759    17442457           19                  11          0   
2            2592   102069605            7                   6          0   
3            1759  1367759732           19                  15          0   
4            1829  2927879268           14                  13          0   
...           ...         ...          ...                 ...        ...   
2231         1765    19708968           18                  10          0   
2232         1759   100731315           13                  12          0   
2233         2570  2399173802           15                   8          0   
2234         1759   380570951            7                  14          0   
2235         2354  3236431915           13                  12          0   

      followers_count  friends_count  listed_count  favourites_count  \
0  

In [27]:
merged_data

,account_age,id,name_length,screen_name_length,protected,followers_count,friends_count,listed_count,favourites_count,geo_enabled,...,1270.2,1271.2,1272.2,1273.2,1274.2,1275.2,1276.2,1277.2,1278.2,1279.2
0,1761,1318284638,17,6,0,28533,1164,45,6006,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1759,17442457,19,11,0,299192,1721,1517,401,1,...,0.0,0.000000,0.000000,0.110568,0.305912,0.000000,0.000000,0.144667,0.071325,0.000000
2,2592,102069605,7,6,0,89051,743,307,154,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1759,1367759732,19,15,0,43211,237,29,358,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1829,2927879268,14,13,0,113300,16422,112,37,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2231,1765,19708968,18,10,0,871041,2531,3771,7581,1,...,0.0,0.062553,0.004812,0.212884,0.411645,0.000000,0.000000,0.659520,0.000000,1.790698
2232,1759,100731315,13,12,0,2956599,75,11604,116,1,...,0.0,0.000000,0.009111,0.171045,0.208733,0.000000,0.031579,0.000000,0.548384,0.067697
2233,2570,2399173802,15,8,0,1035,745,2,0,0,...,0.0,0.000000,0.000000,0.000000,0.464781,1.399824,0.000000,0.000000,0.000000,0.292680
2234,1759,380570951,7,14,0,1732798,709,4355,3770,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# 划分
from sklearn.model_selection import train_test_split
merged_data.columns = merged_data.columns.astype(str)
result_test.columns = result_test.columns.astype(str)
traindata, devdata = train_test_split(merged_data, test_size=0.2, random_state=42)
y_train = traindata['label']
X_train = traindata.drop(columns=['label'])
y_dev = devdata['label']
X_dev = devdata.drop(columns=['label'])

X_train

,account_age,id,name_length,screen_name_length,protected,followers_count,friends_count,listed_count,favourites_count,geo_enabled,...,1270.2,1271.2,1272.2,1273.2,1274.2,1275.2,1276.2,1277.2,1278.2,1279.2
1269,1759,1491180668,4,9,0,1333,769,12,23585,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1532,1763,1085149063,14,12,0,1149308,23217,1323,58621,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
617,2508,727019388,8,14,0,1078,565,30,0,0,...,1.251323,0.000000,0.305250,0.204407,2.156821,1.150250,0.298543,0.218793,0.000000,1.225389
1405,1760,65773523,13,12,0,1228718,500,3724,1338,0,...,0.000000,0.000000,0.000000,0.404217,0.617010,0.008241,0.008390,1.015608,0.000000,1.114699
1134,1760,72378123,10,9,0,800196,25037,1111,2680,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1638,1759,2557928030,6,6,0,580960,465,1201,4774,1,...,0.000000,0.680049,0.000000,0.000000,0.033849,0.027060,0.000000,0.000000,0.030981,1.128738
1095,1760,55631242,15,13,0,1138498,714,3744,542,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1130,1759,14114267,11,11,0,128620,2817,1207,3066,1,...,0.000000,0.000000,0.000000,0.000000,0.004341,0.000000,0.361798,0.000000,0.026009,0.155163
1294,1847,1012481676524577024,9,8,0,1,8,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# 导入必要的库
from sklearn.metrics import accuracy_score, classification_report
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier

# 初始化 HistGradientBoostingClassifier 模型
hist_gbdt_model = HistGradientBoostingClassifier(max_iter=150, learning_rate=0.05, max_depth=3, random_state=42)

# 训练模型
hist_gbdt_model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = hist_gbdt_model.predict(X_dev)

# 评估模型性能
accuracy = accuracy_score(y_dev, y_pred)
print(f'准确率: {accuracy:.2f}')

# 打印分类报告
print('分类报告:\n', classification_report(y_dev, y_pred))


准确率: 0.77
分类报告:
               precision    recall  f1-score   support

           0       0.87      0.60      0.71       205
           1       0.73      0.92      0.82       243

    accuracy                           0.77       448
   macro avg       0.80      0.76      0.76       448
weighted avg       0.79      0.77      0.77       448



In [25]:
result_test = result_test.drop(columns = ['label'])
result_test

KeyError: "['label'] not found in axis"

In [29]:
test_result = hist_gbdt_model.predict(result_test)
test_result = pd.DataFrame(test_result)
test_result

,0
0,1
1,1
2,0
3,1
4,0
...,...
243,0
244,1
245,0
246,0


In [31]:
output = pd.concat([result_test, test_result], axis=1)
output.to_excel(r"C:\Users\17293\Desktop\大三上\社会网络分析\数据处理_new\output.xlsx", index=False)
